In [ ]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from src.GGUFBot import GGUFBot
from src.HFDataset import HFDataset
from src.SimpleQuestionGenerator import SimpleQuestionGenerator
from src.AnswerGenerator import AnswerGenerator
import pandas as pd
import random
import json
from src.GGUFEvaluator import prepare_prompt,parse_output,GGUFEvaluator

In [ ]:

model_path="/home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf"
max_new_tokens=4000
n_layers=400
bot = GGUFBot(model_path, max_new_tokens=max_new_tokens, n_ctx=max_new_tokens, n_gpu_layers=n_layers)

In [ ]:
txt="""次の文章を､日本の文化に即した平易な文章に書き直してください
#文章
次の記事を読んで、最も適切な回答を選択してください。記事: 昔ながらの遊び - 有料の子供向け!ボールを飲みに来てください！あるいは60,000個！アメリカ全土のショッピングモールや近所に、おもちゃやゲーム、ガイド付きの楽しみ、家計に負担をかけないトレーニングなどを特徴とする、従量制の屋内遊び場を提供する新しいタイプのビジネスフランチャイズが登場している。公共の遊び場がますます磨耗して汚くなる中、営利センターでは、清潔で安全なガイド付きアクティビティや、子供の体力を伸ばすためのさまざまなやりがいのあるエクササイズを、通常は 1 時間あたり約 5 ドルの料金で提供しています。カンザスシティに本拠を置くチェーン店、ニューヨーク州ヨンカーズにある設立2か月のディスカバリー・ゾーンのオーナー、ディック・グッゲンハイマー氏は、「遊び場は汚いし、案内もされていない」と語る。安全です」。共働き世帯のニーズを満たすために、この新しいフランチャイズは、従来の公共の遊び場が暗くなって使用できなくなってからもずっと夜も営業しています。ただし、これらの新しい遊び場は保育所を目的としたものではありません。親は子供を送り出すのではなく、家にいて一緒に遊ぶことが期待されています。しかし、ハイテクベビーシッターサービスを提供しているところもあります。ディスカバリー ゾーンの一部では、親が子供を特別なガイド付きプログラムに登録した後、子供を残して数時間こっそりと映画やディナーを楽しむことができます。何か問題があれば、お父さんとお母さんが呼ばれます。しかし、何よりも楽しいのは、共働き家庭で通勤に 2 時間かかる以前の時代に親がやっていたようなこと、子供たちと遊ぶことができるようになることです。少なくとも、時間当たりの料金を考えても、これは時代遅れです。質問: この記事は主に何について話していますか? - お子様は親の世話なしで遊ぶことができます。 - ディスカバリーゾーンの急速な発展。 - 新しいタイプのキッズビジネスフランチャイズ。 - 屋外遊び場の欠点。 この質問に対する答えは次のとおりです
#書き直した文章
"""
txt="""step by stepで､日本語で回答してください
タスクは、与えられた数学の問題に対する正しい答えを生成することです。 Q: 問題: 金は水の 19 倍、銅は水の 9 倍の重さです。水の15倍の重さの合金を得るには、これらをどのような割合で混合すればよいでしょうか? 答え: """
r=bot.ask(txt)
r

In [ ]:
from datasets import load_dataset
ds=load_dataset("atsushi3110/cross-lingual-openorcha-830k-en-ja",split="train")


In [16]:
df=pd.DataFrame(ds[:10])
df["database"]="atsushi3110/cross-lingual-openorcha-830k-en-ja_"+df["id/en"]
df["question"]=df["question/ja"]
df=df.drop(columns=["response/en","system_prompt/en","question/ja","response/ja","question/en","id/en"],axis=1)
df[:3]

,database,question
0,atsushi3110/cross-lingual-openorcha-830k-en-ja...,映画のあらすじから次の質問に対する答えを抽出してください。回答できない質問の場合は「回答でき...
1,atsushi3110/cross-lingual-openorcha-830k-en-ja...,「Anelik ® 送金を利用して入金する」はどうですか？ロシア語で言いましたか？\n
2,atsushi3110/cross-lingual-openorcha-830k-en-ja...,詳細な手順: このタスクでは、提供されたオプションから特定の問題に対して正しいオプションを提...


: 

In [ ]:
from datasets import load_dataset
import random


ds_name="hatakeyama-llm-team/WikiBookJa"
ds=load_dataset(ds_name,split="train")

In [ ]:
inst_list=[
"回答文は丁寧であること"
"回答文は簡潔であること"
"回答文はstep by stepで作文してください",
]

def gen_prompt(inst,text):
    prompt_template=f"""次の文章をもとに､日本語の質問文と日本語の回答文をそれぞれ一つ生成しなさい
    #制約
    - {inst}
    #文章
    {text}
    #質問文と回答文
    """
    return prompt_template

In [ ]:
record=ds[random.randint(0,len(ds))]
prompt=gen_prompt(random.choice(inst_list)[:1000],record["text"])
r=bot.ask(prompt)

In [ ]:
q,a=r.split("回答文")
q=q.replace("質問文","").strip()
a=a.strip()
if a[0]=="：":
    a=a[1:]
if q[0]=="：":
    q=q[1:]
if a[0]==":":
    a=a[1:]
if q[0]==":":
    q=q[1:]

a=a.strip()
q=q.strip()
q,a

In [ ]:
r